In [11]:
from cvxpy import *
import numpy as np

# Problem setup
N = 8  # Number of beam positions
P_max = 250  # Maximum power (W)
noise_power = 1e-10  # Noise power (W)
h = np.random.rand(N, N)  # Channel gain matrix (NxN)
B = 200e6  # Bandwidth (Hz)
T_b = 1e-3  # Time slot duration (s)
alpha = np.random.rand(N)  # Slack parameter alpha

# Variables

# Variables
P = Variable(N, nonneg=True)
constraints = []

# SOCP constraints
for k in range(N):
    interference = sum([h[k, j] * P[j] for j in range(N) if j != k])
    constraints.append(h[k, k] * P[k] >= norm([noise_power, interference]))

constraints.append(sum(P) <= P_max)

# Objective
objective = Maximize(-sum(alpha / P))

# Solve
problem = Problem(objective, constraints)
problem.solve()

print("Optimal Power (SOCP):", P.value)

ValueError: The input must be a single CVXPY Expression, not a list. Combine Expressions using atoms such as bmat, hstack, and vstack.

In [9]:
import cvxpy as cp
import numpy as np

# Problem setup
N = 8  # Number of beam positions
P_max = 250  # Maximum power (W)
noise_power = 1e-10  # Noise power (W)
h = np.random.rand(N, N)  # Channel gain matrix (NxN)
B = 200e6  # Bandwidth (Hz)
T_b = 1e-3  # Time slot duration (s)
alpha = np.random.rand(N)  # Slack parameter alpha

# Variables
P = cp.Variable(N, nonneg=True)  # Transmission power
SINR = cp.Variable(N)

# Constraints
constraints = []
for k in range(N):
    interference = cp.sum([h[k, j] * P[j] for j in range(N) if j != k])
    SINR[k] = h[k, k] * P[k] / (noise_power + interference)
    constraints.append(B * T_b * cp.log(1 + SINR[k]) <= 100 * 1e3)

constraints.append(cp.sum(P) <= P_max)

# Objective function
objective = cp.Maximize(cp.sum(-alpha / SINR))

# Solve
problem = cp.Problem(objective, constraints)
problem.solve()

print("Optimal Power:", P.value)

TypeError: 'Variable' object does not support item assignment

In [12]:
from scipy.optimize import minimize

# Lagrangian function
def lagrangian(P, lambda_):
    interference = np.array([np.sum(h[k, :] * P) - h[k, k] * P[k] for k in range(N)])
    SINR = h.diagonal() * P / (noise_power + interference)
    obj = -np.sum(alpha / SINR) + lambda_ * (np.sum(P) - P_max)
    return obj

# Optimize
lambda_ = 1  # Initial Lagrange multiplier
res = minimize(lambda P: lagrangian(P, lambda_), np.ones(N), bounds=[(0, None)] * N)
print("Optimal Power:", res.x)

Optimal Power: [1. 1. 1. 1. 1. 1. 1. 1.]


C:\Users\flyvideo\AppData\Local\Temp\ipykernel_17596\2136584790.py:7: RuntimeWarning: divide by zero encountered in divide
  obj = -np.sum(alpha / SINR) + lambda_ * (np.sum(P) - P_max)
e:\Anaconda3\envs\spin\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
